# 2. Experimentation Basics: Additional Popular Tests Part 1

*note: if you see any mistakes, please feel free to let me know so that I can improve the notebook!*

This notebook is a sequel to [1. basics](https://github.com/kideokkwon/experimentation-simulation-and-text-notes/blob/main/simulation/1_basics_standard_abtest_example.ipynb).

While the most common type of test statistic you will run into when performing A/B tests is the two-proportion $Z$-test, it is worthwhile to cover some of the other "basic" test statistics for slightly different scenarios before diving into A/B test concepts further.

In this notebook, we will talk about the one-sample $Z$-test and the Binomial Test. In the next notebooks we may cover more popular tests and then we will move onto the concept of Power and Power Analysis.

## One-Sample $Z$-test procedure

Last notebook we went over the two-proportion $Z$-test, which is likely the most "common" type of test you will run into as an analyst performing A/B tests. However, the most commonly taught variation of the $Z$-test in the classroom setting is likely the one-sample $Z$-test, where you are testing if the population mean $\mu$ equals from hypothetical value $\mu_0$.

The hypothesis is written as follows

$$\begin{align} H_0&:\mu=\mu_0\\ H_1&:\mu\neq\mu_0 \end{align}$$

You also choose the threshold $\alpha$. Like with hypothesis tests in general, we then take the formula of the appropriate test statistic (which is a function of the *observed* data), which in this case is

$$Z=\frac{\bar x-\mu_0}{\sigma/\sqrt{n}}$$

Notice how there are 4 bits of information you have to plug in
- $\mu_x$: the mean of your observed data
- $\mu_0$: the prespecified value you want to test in the hypothesis
- $n$: the sample size of your observed data
- $\sigma$: the population standard deviation (eh?!)

Then, assuming this is a two-sided test, you compute the $p$-value using the formula

$$p=2\times\Phi(Z\geq |z|)$$

where $z$ is the observed value of the test statistic $Z$ and $\Phi(\cdot)$ is the cdf of the standard normal distribution. Then, you reject the null if $p<\alpha$.

Before we run through an example, we need to clarify the conditions in which you run this test.

First, remember that the test is run to test if the population mean of what you are studying equals some certain value $\mu_0$.

Technically speaking, we also require the following assumptions:
- your data needs to be continuous
- your data follows a normal distribution
- Random sampling
- Population standard deviation is known

This can be confusing because these conditions feel very limiting. For example, the population standard deviation doesn't sound like something that is often known, so what do we do? (For the normal distribution assumption, due to the central limit theorem, this is not an issue if the data is sufficiently large)

Let's first proceed with a classroom-esque example that doesn't really make much sense either (to demonstrate the above with actual numbers), and then afterwards we will consider a very practical scenario where we can utilize the $Z$-test so that we can show how there are plenty of scenarios where this test is useful.

## One-sample $Z$-test; classroom-esque example

This example is taken directly from [this link](https://statisticsbyjim.com/hypothesis-testing/z-test/).

Suppose we randomly sample subjects from an honors program. We want to know if their average IQ differs from the general population. Let's say that we also know that the average IQ of the population is 100 with a standard deviation of 15. We will perform a two-sided test with $\alpha=0.05$.

$\begin{align}H_0&:\mu=100\\H_1&:\mu\neq 100\end{align}$

In our random sample of honors students, we have the following information:
- average IQ of sample: $\bar x = 107$
- sample size: $n=25$

Now we have everything to fill out $Z=\frac{\bar x-\mu_0}{\sigma/\sqrt{n}}$. We already know $\mu_0$ and we know $\sigma$ because the standard deviation for the general population is $15$. That may cause confusion because that isn't the standard deviation of honors students, but remember that hypothesis tests are (stochastic) proofs by contradiction; we are assuming that the null is true and that there is no difference in IQ among honors students and the general population.

Plugging in the numbers, we get $\frac{107-100}{15/\sqrt{25}}=\frac{7}{3}\approx 2.33$.

Then, we can calculate the $p$-value below:



In [1]:
import scipy.stats as st
z = 7/3
p_value = 2*(1 - st.norm.cdf(abs(z)))
print(p_value)

0.01963065725729063


Because the $p$-value is below the $\alpha=0.05$ threshold, we would reject the null hypothesis and conclude for the purpose of the study that on average, there may be evidence to show that the IQ of honors students (from the population of honor students we sampled from) are higher than the general population.

## One-sample $Z$-test and the Binomial Test; practical example

Let's say that your friend starting his first programming class wrote you a coin flipper app and you flipped it $100$ times for fun and got $60$ heads and $40$ tails. You become a little suspicious because you consider your friend to be a frequent prankster and hypothesize that he may have rigged it to not be 50/50 (but you do believe that the independent probability $p$ is the only thing he rigged and otherwise it functions as expected of a coin flipper). You decide to do a hypothesis test to see if there's some evidence to show that it is not 50/50.

$$\begin{align}H_0&:p=0.5\\H_1&:p\neq 0.5\end{align}$$

We know that the distribution of # of heads in $n$ independent flips with a fixed probability $p$ is modeled by a Binomial distribution ($X\sim\text{Binomial}(n,p)$).

We also know that as the sample size grows, we can approximate $\text{Binomial}(n,p)$ with a normal distribution via

$$\mathcal{N}(np,np(1-p))$$

The cool part about this is that now we know the standard deviation (under the null) as well (because we know the standard deviation of a binomial design), so we *do* know $\sigma$.

Since we are interested in the proportion $p$, we scale $X$ with $X/n$, which means that our normal approximation is

$$\mathcal{N}\left(p,\frac{p(1-p)}{n}\right)$$

If you are interested in how the variance changes to have $n$ in the denominator, read this [comment](https://stats.stackexchange.com/a/85827/391916).

Now remember that the way we get the test statistic is to assume the null and standardize it so that we can utilize the well-known cdf of the standard normal distribution. Thus, the test statistic is
$$Z=\frac{\hat p-p}{\sqrt{\frac{p(1-p)}{n}}}=\frac{\frac{60}{100}-0.5}{\sqrt{0.5(0.5)/100}}=2$$

Then, we calculate the $p$-value below:

In [2]:
import scipy.stats as st
z = 2
p_value = 2*(1 - st.norm.cdf(abs(z)))
print(p_value)

0.04550026389635842


Given the $p$-value in relation to the typical alpha of $\alpha=0.05$, you might then conclude that the app *may* be rigged, thus leading to a confrontation with your friend.

One question that may come about is - what if we just did a Binomial test? Why rely on the normal approximation? One reason that is cited to rely on the normal approximation is computational speed, especially with larger sample sizes. But our sample size is fairly low so we can check out the Binomial test too.

We'll do that below. You can read more about this [here](https://en.wikipedia.org/wiki/Binomial_test).

Let's say that we have a null hypothesis that the probability of getting tails is $0.5$, with our alternative hypothesis being that it is greater than $0.5$, meaning that this is a one-sided test, meaning you don't even consider the possibility that it's lower than $0.5$. Perhaps not realistic, but two-sided tests will be covered later because they are more complex than one-sided for binomial distributions.

$$\begin{align}H_0&:p=0.5\\H_1&:p > 0.5\end{align}$$

The binomial test is an *exact* test because we calculate the exact $p$-value. I would say that this term isn't exactly the most intuitive, but as an example, let's say you flipped a coin twice and got tails both times. The $p$-value is the probability of getting that result or more extreme. Note how in the distribution of # of heads/tails after 2 flips - there are only 4 equally-likely scenarios: TT, HH, HT, and TH. Thus, the probability of getting 2 tails or more is just $1/4=0.25$, which is the *exact* $p$-value (for a one-sided test). Typically you would not reject the null that the coin is fair (50/50 independent chance to get heads or tails) when given a $p$-value of $0.25$, and this should line up with your intuition as well - you would not be suspicious of a coin that claims to be fair just because you got two tails in a row.

The binomial test just generalizes this to $n$ samples instead of $2$. Notice how in this scenario, each sample is independent, which you can think of sampling from a pool with replacement. In the case that there is no replacement, the hyper-geometric exact test may be more relevant. This hyper-geometric exact test is actually one of the earliest tests demonstrated by Ronald A. Fisher in his 1935 text on testing, through a famous example commonly referred to as "Fisher's Lady Tasting Tea".

Going back to our binomial test - to get the probability of getting $k$ heads after $n$ flips, we use the binomial probability mass function

$$\Pr(X=k)={n\choose k}p^k(1-p)^{n-k}$$

Of-course, in our test setup, $p=0.5$ because we calculate the $p$-value while assuming that the null is true.

This isn't enough to calculate the $p$-value, because we don't just get the probability of getting $k$ out of $n$ as tails. We need $k$ *or more extreme*, so we have to add up all the possible probabilities of getting $k,k+1,...,n$ tails (if you flip a coin $n$ times, the most tails you can get is $n$). This can be represented as

$$p=\sum_{i=k}^{n}\Pr(X=i)=\sum_{i=k}^{n}{n\choose i}p^i(1-p)^{n-i}$$

Now, if our alternative hypothesis was $H_1:p < 0.5$ instead, then we would be summing from $0$ to $k$.

You may be wondering, "what about two-sided tests? Can we just double the $p$-value like with the normal distribution?". Not exactly, because if $p\neq 0.5$, the binomial distribution is not symmetric (although in our case it is).

Instead, remember that the $p$-value is the probability of getting your result or more extreme, meaning that they are less likely. So we just gather all the hypothetical success probabilities that are less likely than (or equal to) the probability of your observed # of success, $k$. This amounts to defining $\mathcal{I}=\{i:\Pr(X=i)\leq\Pr(X=k)\}$ and then applying the formula

$$p=\sum_{i\in\mathcal{I}}\Pr(X=i)=\sum_{i\in\mathcal{I}}{n\choose i}p^i(1-p)^{n-i}$$

Next, let's show how this two-sided test can be done on Python, one using `scipy` and another creating our own function.


In [3]:
import scipy.stats as st
p_value = st.binomtest(60, 100, 0.5, alternative='two-sided')
print(p_value)

BinomTestResult(k=60, n=100, alternative='two-sided', statistic=0.6, pvalue=0.056887933640980784)


The $p$-value is relatively close to the normal distribution one. Of-course, it isn't the same because the normal distribution is an approximation. In a future notebook we will compare and see how it converges as $n$ increases. But first, let's try to code the above ourselves.

In [4]:
import numpy as np
from scipy.special import comb

def binomial_pmf(n, k, p):
    """
    Computes the binomial probability at X=k.

    Args:
        n (int): The sample size.
        k (int): The number of successes.
        p (float): The probability of success.

    Returns:
        float: The binomial probability at X=k.
    """

    return comb(n, k) * p**k * (1 - p)**(n - k)

def binom_test(n, k, p):
    """
    Computes the two-sided binomial test.

    Args:
        n (int): The sample size.
        k (int): The number of successes.
        p (float): The probability of success.

    Returns:
        float: The two-sided binomial test p-value.
    """

    pmf_dict = {}
    for i in range(0, n + 1):
        pmf_dict[i] = binomial_pmf(n, i, p)

    sorted_pmf_dict = sorted(pmf_dict.items(), key=lambda x: x[1])

    p_value = 0
    for i, pmf in sorted_pmf_dict:
        if pmf_dict[i] > pmf_dict[k]:
            break
        p_value += pmf

    return p_value

In [5]:
binom_test(n=100,k=60,p=0.5)

0.056887933640980784

We obtained the exact same result by copying the mathematical formula given above. Notice how the $p$-value is a little higher. Now, since the normal approximation is an *approximation*, you might rely on the binomial result more, meaning that given the standard $\alpha=0.05$, you won't reject the null. How does this line up with your intuition? Would you be suspicious of a coin that landed tails 60 times out of a 100?